In [ ]:
%%time
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
from gensim.models import KeyedVectors


pt_w2v_patch = 'data/word_embedding/cbow_s300.txt'
pt_w2v = KeyedVectors.load_word2vec_format(pt_w2v_patch, unicode_errors="ignore")
pt_model_w2v = {w: vec for w, vec in zip(pt_w2v.index2word, pt_w2v.syn0)}

In [1]:
import datasets_manager as DM

chave = DM.chave_questions()

ret = []
for question in chave:
    if question['question'] is None or question['question'].strip() == '':
        continue
    ret.append(question)
chave = ret

In [2]:
X, y = DM.separete_questions(chave)
X = DM.words2indexes(X)

In [11]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_testv = train_test_split(X, y, test_size=0.2)

In [26]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=8000)


In [48]:
aux = tokenizer.sequences_to_matrix(x_train)
s = 8
print(sum(aux[s]))
print(len(x_train[s]))

6.0
6


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer


cv = CountVectorizer(analyzer='word', strip_accents=None)

cv = cv.fit(x_train)

In [ ]:
cv.inverse_transform([cv.build_tokenizer()(x_train[0])])

In [ ]:
cv.build_tokenizer()(x_train[0])

In [ ]:
def w2v_layer(w2v_vocabulary, w2v_dimensions, w2v_weights):
    w2v_input = Input(shape=(None,), dtype='int32', name='w2v_input')
    words = Embedding(input_dim=w2v_vocabulary, output_dim=w2v_dimensions,  weights=[w2v_weights], trainable=False)(w2v_input)

def bow_layer():
    pass

from keras.layers import Input, Dense, Embedding, Conv2D, MaxPool2D
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.models import Model


sequence_length = x.shape[1] # 56
vocabulary_size = len(vocabulary_inv) # 18765
embedding_dim = 256
filter_sizes = [3,4,5]
num_filters = 512
drop = 0.5

epochs = 100
batch_size = 30

inputs = Input(shape=(sequence_length,), dtype='int32')
embedding = Embedding(input_dim=vocabulary_size, output_dim=embedding_dim, input_length=sequence_length)(inputs)
reshape = Reshape((sequence_length, embedding_dim, 1))(embedding)

conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
conv_2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)

maxpool_0 = MaxPool2D(pool_size=(sequence_length - filter_sizes[0] + 1, 1), strides=(1,1), padding='valid')(conv_0)
maxpool_1 = MaxPool2D(pool_size=(sequence_length - filter_sizes[1] + 1, 1), strides=(1,1), padding='valid')(conv_1)
maxpool_2 = MaxPool2D(pool_size=(sequence_length - filter_sizes[2] + 1, 1), strides=(1,1), padding='valid')(conv_2)

concatenated_tensor = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2])
flatten = Flatten()(concatenated_tensor)
dropout = Dropout(drop)(flatten)
output = Dense(units=2, activation='softmax')(dropout)

# this creates a model that includes
model = Model(inputs=inputs, outputs=output)

checkpoint = ModelCheckpoint('weights.{epoch:03d}-{val_acc:.4f}.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
print("Traning Model...")
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, callbacks=[checkpoint], validation_data=(X_test, y_test))  # starts training



